## 賃料をlog, 1平米あたりの価格をlightgbmで予測して、あとから面積のlogをたす

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib
import pickle

In [2]:
df=pd.read_csv('preprocessed_data.csv')
df.head()

,カテゴリ,建物名,築年数,階数,階,賃料,管理費,敷金,礼金,間取り,専有面積,23区,都市名,路線,最寄駅,徒歩
0,賃貸アパート,プチ・ボヌール,4,2,2,148000.0,5000.0,148000.0,148000.0,42DK,48.87,大田区,南蒲田３,京急空港線,糀谷駅,7
1,賃貸一戸建て,荒川戸建,10,3,3,195000.0,0.0,390000.0,195000.0,3SLDK,97.95,荒川区,荒川２,東京メトロ千代田線,町屋駅,5
2,賃貸一戸建て,荒川戸建,10,3,3,195000.0,0.0,390000.0,195000.0,3SLDK,97.95,荒川区,荒川２,東京メトロ千代田線,町屋駅,5
3,賃貸マンション,ティモーネ上野の森ノース,5,11,2,152000.0,10000.0,152000.0,152000.0,2LDK,52.04,荒川区,東日暮里４,ＪＲ山手線,鶯谷駅,6
4,賃貸アパート,東京メトロ日比谷線 三ノ輪駅 3階建 築13年,13,3,3,113000.0,2000.0,113000.0,113000.0,1LDK,43.81,荒川区,南千住５,東京メトロ日比谷線,三ノ輪駅,5


In [3]:
###カテゴリカル変数に変換
df['カテゴリ']=df['カテゴリ'].astype('category') 
df['間取り']=df['間取り'].astype('category')
df['23区']=df['23区'].astype('category')
df['都市名']=df['都市名'].astype('category')
df['路線']=df['路線'].astype('category')
df['最寄駅']=df['最寄駅'].astype('category')

In [9]:
print(10/100)
print(10**np.log10(10/100))
print(10**(np.log10(10)-np.log10(100)))

0.1
0.1
0.1


In [16]:
print((10/100)*100)
print(10**(np.log10(10)-np.log10(100)+np.log10(100)))

10.0
10.0


In [7]:
###賃料のlog
df['log_賃料']=np.log(df['賃料'])

###面積のlog
df['log_面積']=np.log(df['専有面積'])

###1平米あたりの賃料
df['log_賃料/面積']=df['log_賃料']-df['log_面積']

df.head()

,カテゴリ,建物名,築年数,階数,階,賃料,管理費,敷金,礼金,間取り,専有面積,23区,都市名,路線,最寄駅,徒歩,log_賃料,log_面積,log_賃料/面積
0,賃貸アパート,プチ・ボヌール,4,2,2,148000.0,5000.0,148000.0,148000.0,42DK,48.87,大田区,南蒲田３,京急空港線,糀谷駅,7,11.904968,3.889164,8.015804
1,賃貸一戸建て,荒川戸建,10,3,3,195000.0,0.0,390000.0,195000.0,3SLDK,97.95,荒川区,荒川２,東京メトロ千代田線,町屋駅,5,12.180755,4.584457,7.596298
2,賃貸一戸建て,荒川戸建,10,3,3,195000.0,0.0,390000.0,195000.0,3SLDK,97.95,荒川区,荒川２,東京メトロ千代田線,町屋駅,5,12.180755,4.584457,7.596298
3,賃貸マンション,ティモーネ上野の森ノース,5,11,2,152000.0,10000.0,152000.0,152000.0,2LDK,52.04,荒川区,東日暮里４,ＪＲ山手線,鶯谷駅,6,11.931636,3.952013,7.979623
4,賃貸アパート,東京メトロ日比谷線 三ノ輪駅 3階建 築13年,13,3,3,113000.0,2000.0,113000.0,113000.0,1LDK,43.81,荒川区,南千住５,東京メトロ日比谷線,三ノ輪駅,5,11.635143,3.779862,7.855281


In [17]:
print(148000/48.87)
print(np.exp(8.015804))
print(np.exp(8.015804+3.889164))

3028.4428074483326
3028.443287565006
148000.0661935695


In [15]:
df.columns

Index(['カテゴリ', '建物名', '築年数', '階数', '階', '賃料', '管理費', '敷金', '礼金', '間取り', '専有面積',
       '23区', '都市名', '路線', '最寄駅', '徒歩', 'log_賃料', 'log_面積', 'log_賃料/面積'],
      dtype='object')

In [18]:
#目的変数と説明変数に分割
X_columns=['カテゴリ', '築年数', '階数', '階', '間取り','23区', '都市名', '路線', '最寄駅', '徒歩','log_面積']
y_columns=['log_賃料/面積']

X=df[X_columns]
y=df[y_columns]

In [24]:
X

,カテゴリ,築年数,階数,階,間取り,23区,都市名,路線,最寄駅,徒歩,log_面積,log_賃料/面積
0,賃貸アパート,4,2,2,42DK,大田区,南蒲田３,京急空港線,糀谷駅,7,3.889164,8.015804
1,賃貸一戸建て,10,3,3,3SLDK,荒川区,荒川２,東京メトロ千代田線,町屋駅,5,4.584457,7.596298
2,賃貸一戸建て,10,3,3,3SLDK,荒川区,荒川２,東京メトロ千代田線,町屋駅,5,4.584457,7.596298
3,賃貸マンション,5,11,2,2LDK,荒川区,東日暮里４,ＪＲ山手線,鶯谷駅,6,3.952013,7.979623
4,賃貸アパート,13,3,3,1LDK,荒川区,南千住５,東京メトロ日比谷線,三ノ輪駅,5,3.779862,7.855281
...,...,...,...,...,...,...,...,...,...,...,...,...
222918,賃貸一戸建て,51,2,2,3K,江戸川区,江戸川２,都営新宿線,瑞江駅,14,3.967836,7.370736
222919,賃貸マンション,37,7,7,1DK,豊島区,高田１,東京メトロ副都心線,雑司が谷駅,4,3.259634,8.169910
222920,賃貸アパート,20,2,1,ワンルーム,品川区,南大井５,京急本線,立会川駅,7,3.352007,7.998399
222921,賃貸マンション,45,3,3,1DK,葛飾区,東新小岩１,ＪＲ総武線,新小岩駅,3,3.413126,7.812117


In [19]:
#学習用、検証用、テスト用に分割
from sklearn.model_selection import train_test_split

X_train_val, X_test, y_train_val, y_test=train_test_split(X, y, test_size=0.2, random_state=20221014)
X_train, X_val, y_train, y_val=train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=20221014)

In [23]:
"""
OptunaでlightGBM
"""

import lightgbm as lgb
import optuna.integration.lightgbm as lgb_o

params={
    "objective": "regression",
    "metric": "l1",
    "n_estimators": 10000
}

reg_train=lgb.Dataset(X_train, y_train)
reg_val=lgb.Dataset(X_val, y_val)

opt=lgb_o.train(params, reg_train, valid_sets=reg_val, num_boost_round=10000, early_stopping_rounds=100, verbose_eval=False)

[I 2022-10-16 22:24:54,073] A new study created in memory with name: no-name-4d6124f5-7aac-4b4c-aa79-5cfe18fc03bd
feature_fraction, val_score: inf:   0%|                   | 0/7 [00:00<?, ?it/s]/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGB

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001902 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3813
[LightGBM] [Info] Number of data points in the train set: 142670, number of used features: 12
[LightGBM] [Info] Start training from score 8.126077


/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')
feature_fraction, val_score: 0.002562:  14%|7    | 1/7 [01:48<10:49, 108.24s/it]/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instea

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003003 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3813
[LightGBM] [Info] Number of data points in the train set: 142670, number of used features: 12
[LightGBM] [Info] Start training from score 8.126077


/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')
feature_fraction, val_score: 0.001434:  29%|#7    | 2/7 [03:12<07:50, 94.19s/it]/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instea

...

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002824 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3813
[LightGBM] [Info] Number of data points in the train set: 142670, number of used features: 12
[LightGBM] [Info] Start training from score 8.126077


/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')
num_leaves, val_score: 0.001081:   5%|5         | 1/20 [02:24<45:44, 144.45s/it]/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instea

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003332 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3813
[LightGBM] [Info] Number of data points in the train set: 142670, number of used features: 12
[LightGBM] [Info] Start training from score 8.126077


/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


KeyboardInterrupt: 

In [ ]:
best_params=opt.params
best_params

In [ ]:
lgb_model=lgb.train(best_params, reg_train, valid_sets=[reg_train, reg_val], verbose_eval=100)